# Exploratory Analysis

## Import

In [4]:
import psycopg2
from psycopg.rows import dict_row
import pandas as pd

## Definitions

In [5]:
def get(query):
    try:
        conn = psycopg2.connect(
            host="localhost",
            database="bronze",
            user="postgres",
            password="postgres",
            )
        cur = conn.cursor()
        cur.execute(query)
        colnames = [desc[0] for desc in cur.description]
        rows = cur.fetchall()
        db =  pd.DataFrame(rows, columns=colnames)
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
    return db

## Tests

In [3]:
query = '''
    SELECT * FROM relatorio_urna
    WHERE nr_votavel = 13
    --GROUP BY cd_carga_1_urna_efetivada
    limit 100000'''

In [6]:
db = get(query)

In [7]:
db.describe().T

,count,mean,std,min,25%,50%,75%,max
ano_eleicao,100000.0,2.022000e+03,0.000000,2022.0,2022.0,2022.0,2022.00,2022.0
cd_tipo_eleicao,100000.0,0.000000e+00,0.000000,0.0,0.0,0.0,0.00,0.0
cd_pleito,100000.0,4.070000e+02,0.000000,407.0,407.0,407.0,407.00,407.0
nr_turno,100000.0,2.000000e+00,0.000000,2.0,2.0,2.0,2.00,2.0
cd_eleicao,100000.0,5.450000e+02,0.000000,545.0,545.0,545.0,545.00,545.0
cd_municipio,100000.0,4.770306e+04,38731.662442,19.0,12068.0,31054.0,91677.00,99430.0
nr_zona,100000.0,2.411726e+01,25.597098,1.0,6.0,16.0,33.00,134.0
nr_secao,100000.0,2.042019e+02,227.669794,1.0,57.0,126.0,280.00,3394.0
nr_local_votacao,100000.0,1.334440e+03,411.692616,1015.0,1058.0,1163.0,1465.00,4510.0
cd_cargo_pergunta,100000.0,1.000000e+00,0.000000,1.0,1.0,1.0,1.00,1.0


## Data Engineering

### Merging tables

In [8]:
query = '''
    SELECT indicador_idhm.localidade, 
        indicador_idhm.idhm, 
        indicador_densidade_demografica.densidade_demografica, 
        indicador_pib_per_capita.pib_per_capita, 
        indicador_populacao.populacao, 
        indicador_populacao_estimada.populacao_estimada, 
        indicador_salario_medio.salario_medio, 
        indicador_taxa_escolarizacao.taxa_escolarizacao,
        indicador_taxa_mortalidade_infantil.taxa_mortalidade_infantil
    FROM indicador_idhm
    INNER JOIN indicador_densidade_demografica
        ON indicador_idhm.localidade = indicador_densidade_demografica.localidade
    INNER JOIN indicador_pib_per_capita
        ON indicador_idhm.localidade = indicador_pib_per_capita.localidade
    INNER JOIN indicador_populacao
        ON indicador_idhm.localidade = indicador_populacao.localidade
    INNER JOIN indicador_populacao_estimada
        ON indicador_idhm.localidade = indicador_populacao_estimada.localidade
    INNER JOIN indicador_salario_medio
        ON indicador_idhm.localidade = indicador_salario_medio.localidade
    INNER JOIN indicador_taxa_escolarizacao
        ON indicador_idhm.localidade = indicador_taxa_escolarizacao.localidade
    INNER JOIN indicador_taxa_mortalidade_infantil
        ON indicador_idhm.localidade = indicador_taxa_mortalidade_infantil.localidade
    '''
db = get(query)

In [9]:
db

,localidade,idhm,densidade_demografica,pib_per_capita,populacao,populacao_estimada,salario_medio,taxa_escolarizacao,taxa_mortalidade_infantil
0,110001,0.641,3.45,25091.18,24392,22516,1.9,95.7,6.01
1,110002,0.702,20.41,25730.20,90353,111148,2.0,97.2,7.30
2,110003,0.650,4.80,32226.25,6313,5067,1.8,98.1,28.99
3,110004,0.718,20.72,29331.29,78574,86416,1.9,97.6,16.60
4,110005,0.692,6.12,37069.25,17029,16088,2.0,98.5,7.43
...,...,...,...,...,...,...,...,...,...
5549,352430,0.778,101.42,40825.26,71662,78029,2.7,97.9,9.30
5550,352440,0.777,454.94,59957.57,211214,237119,3.0,98.3,9.73
5551,352450,0.723,38.87,77724.70,5657,7322,2.3,97.7,9.52
5552,352460,0.717,24.44,29046.48,17208,17911,1.9,97.4,12.61
